In [226]:
from langgraph.graph import StateGraph, START, END 
from typing_extensions import TypedDict
from langchain.chat_models import init_chat_model
# from langchain_core.messages import AnyMessage
from langgraph.graph.message import MessagesState #, add_messages

llm = init_chat_model("openai:gpt-4o-mini")

In [227]:
from typing import Annotated


class State(MessagesState):
    # messages: Annotated[list[AnyMessage], add_messages]
    custom_stuff: str
    
    
graph_builder = StateGraph(State)

In [228]:
def chatbot(state: State):
    response = llm.invoke(state["messages"])
    return {
        "messages": [response]
    }

In [229]:
graph_builder.add_node("chatbot", chatbot)

graph_builder.add_edge(START, "chatbot")
graph_builder.add_edge("chatbot", END)

In [230]:
graph = graph_builder.compile()
# graph
print(graph.get_graph().draw_ascii())

+-----------+  
| __start__ |  
+-----------+  
      *        
      *        
      *        
 +---------+   
 | chatbot |   
 +---------+   
      *        
      *        
      *        
 +---------+   
 | __end__ |   
 +---------+   


In [231]:
print(graph.invoke({"messages": [{"role": "user", "content": "how are you ?"}]}))


{'messages': [HumanMessage(content='how are you ?', additional_kwargs={}, response_metadata={}, id='154b96ed-2e5b-48fe-b09a-9e69a397b642'), AIMessage(content="I'm just a computer program, so I don't have feelings, but I'm here and ready to help you! How can I assist you today?", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 28, 'prompt_tokens': 11, 'total_tokens': 39, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_51db84afab', 'id': 'chatcmpl-CKlJX1kSQF8bxDiZxR3pHcsBY56IC', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--62753cb1-07e7-495a-a0d9-0dc7b194b5c4-0', usage_metadata={'input_tokens': 11, 'output_tokens': 28, 'total_tokens': 39, 'input_token_details': {'audio': 0, 'cache_read': 0